In [1]:
import pandas as pd
import numpy as np

In [2]:
# Cargamos los datos
data = pd.read_csv('../datasets/competencia_01_crudo.csv')

In [2]:
# Trabajamos solamente con las columnas necesarias
clientes = data[['numero_de_cliente', 'foto_mes']].copy().sort_values(['numero_de_cliente', 'foto_mes'])

# Convertimos foto mes a un periodo numérico secuencial
clientes['periodo0'] = clientes['foto_mes'].floordiv(
    100).mul(12) + clientes['foto_mes'].mod(100)

# Calculamos el último y anteúltimo periodo
ultimo = clientes['periodo0'].max()
anteultimo = ultimo - 1

In [14]:
# Creamos los leads de período según cliente
groupper = clientes.groupby('numero_de_cliente')['periodo0']
clientes['periodo1'] = groupper.shift(-1)
clientes['periodo2'] = groupper.shift(-2)

In [40]:
# Si el cliente tiene vacío periodo 1, es BAJA+1, si tiene vacío periodo 2, es BAJA+2
clientes['clase_ternaria'] = np.where(clientes['periodo0'] == ultimo, pd.NA,
                                      np.where(clientes['periodo1'].isna(), 'BAJA+1',
                                               np.where(clientes['periodo0'] == anteultimo, pd.NA,
                                                        np.where(clientes['periodo2'].isna(), 'BAJA+2', 'CONTINUA'))))

In [39]:
data.join(clientes['clase_ternaria']).to_csv('../datasets/competencia_01.csv', index=False)

In [36]:
clientes.loc[clientes['periodo0'] < anteultimo, 'clase_ternaria'].value_counts(dropna=False, normalize=True).mul(100).round(2)

CONTINUA    98.87
BAJA+2       0.61
BAJA+1       0.52
Name: clase_ternaria, dtype: float64

## Test biwtise

In [3]:

foto_mes_array = data['foto_mes'].to_numpy()
numero_de_cliente_array = data['numero_de_cliente'].to_numpy()

months = np.sort(np.unique(foto_mes_array))

client_sets_df = pd.DataFrame({
    'month': months,
    'clients': [set(numero_de_cliente_array[foto_mes_array == month]) for month in months]
})

for i in range(len(months) - 2):
    period_0 = client_sets_df.loc[client_sets_df['month'] == months[i], 'clients'].values[0]
    period_1 = client_sets_df.loc[client_sets_df['month'] == months[i + 1], 'clients'].values[0]
    period_2 = client_sets_df.loc[client_sets_df['month'] == months[i + 2], 'clients'].values[0]

    BAJA_2 = period_0 - period_1 - period_2
    BAJA_1 = (period_0 & period_1) - period_2

    data['clase_ternaria'] = 0
    data.loc[(data['foto_mes'] == months[i]) & (data['numero_de_cliente'].isin(BAJA_1)), 'clase_ternaria'] = 1
    data.loc[(data['foto_mes'] == months[i]) & (data['numero_de_cliente'].isin(BAJA_2)), 'clase_ternaria'] = 2


In [11]:
data.query('numero_de_cliente == 250971063').sort_values('foto_mes')[['foto_mes', 'clase_ternaria']]

,foto_mes,clase_ternaria
356,202101,0
162380,202102,0
325023,202103,0
488709,202104,1
652799,202105,0


In [7]:
data.query('clase_ternaria != 0')

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
488414,249458924,202104,1,0,0,57,128,2441.27,16046.58,2666.93,...,9.0,0.00,-20762.10,0.0,3720.0,1499.09,1.0,0.0,1747.77,1
488602,250352496,202104,1,1,0,71,323,561.83,65180.25,941.59,...,2.0,0.00,-83623.00,0.0,8927.0,7856.24,4.0,0.0,1583.55,2
488675,250776861,202104,1,0,0,65,323,398.71,19308.91,162.35,...,23.0,30743.52,-35066.28,0.0,8920.0,11807.26,2.0,0.0,1571.82,2
488709,250971063,202104,1,0,1,56,206,5153.05,22885.20,2467.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
488755,251168989,202104,1,0,0,53,284,2911.80,19062.48,2249.08,...,2.0,0.00,-2958.95,0.0,7766.0,0.00,0.0,0.0,0.00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651927,1559354136,202104,1,0,0,65,5,248.03,2939.29,8.69,...,2.0,0.00,-8764.25,0.0,138.0,0.00,0.0,0.0,0.00,2
652094,1573517883,202104,1,0,0,27,5,-5781.15,-5377.92,83.64,...,2.0,0.00,0.00,0.0,131.0,2476.20,2.0,0.0,0.00,2
652119,1574298927,202104,1,0,0,29,3,328.52,696.62,184.27,...,2.0,0.00,0.00,0.0,85.0,2275.62,2.0,0.0,117.30,2
652340,1588135381,202104,0,0,0,30,1,0.00,0.00,0.00,...,2.0,0.00,NaN,NaN,23.0,NaN,NaN,NaN,0.00,2


In [12]:
client_sets_df

,month,clients
0,202101,"{578813954, 454033415, 570949644, 835190797, 1..."
1,202102,"{578813954, 454033415, 570949644, 835190797, 1..."
2,202103,"{578813954, 454033415, 570949644, 835190797, 1..."
3,202104,"{578813954, 570949644, 835190797, 845152269, 1..."
4,202105,"{578813954, 570949644, 835190797, 845152269, 1..."
5,202106,"{578813954, 570949644, 835190797, 845152269, 1..."
